In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import re
import os

date = datetime.today().strftime("%Y년 %m월 %d일 뉴스입니다.\n")

query = input('검색할 키워드를 입력하세요 :')
#띄어쓰기 입력시 URL 조건절에 +로 입력되므로
query = query.replace(' ','+')

news_num = int(input('필요한 뉴스기사 수를 입력해주세요 : '))

news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

req = requests.get(news_url)
soup = BeautifulSoup(req.text, 'html.parser')

#뉴스 번호
idx = 0
#검색 페이지
cur_page = 1

while idx < news_num:
    #a 태그, news_tit 클래스
    a_list = soup.findAll('a','news_tit')

    #요청한 뉴스의 개수가 많은 경우를 대비
    #개수가 적으면 가져온 데이터 중 일부만 출력
    for n in a_list[:min(len(a_list),news_num-idx)]:
        print(str(idx+1)+'번째 뉴스: ' + n.get('title'))
        idx+=1
    cur_page +=1

    pages = soup.find('div', 'sc_page_inner')
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')



검색할 키워드를 입력하세요 :코로나
필요한 뉴스기사 수를 입력해주세요 : 3
1번째 뉴스: 노바백스 코로나 백신 9일 출하, 10일 접종계획 발표
2번째 뉴스: "코로나19 환자 진료 병·의원, 포털사이트에서 검색하세요"
3번째 뉴스: 식약처, 노바백스 코로나 백신 84만 회분 국가출하승인


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas

def get_movie_point(start,end):
    result = []

    for i in range(start,end+1):
        print(f"page: {i}")
        url = f"https://movie.naver.com/movie/point/af/list.naver?&page={i}"
        response = requests.get(url)
        bs = BeautifulSoup(response.text, "html.parser")

        #부모부터 자식까지 접근하기
        #통째로 묶은 table 태그의 list_netizen 클래스로 접근
        table = bs.find("table","list_netizen")

        #그 중 tr태그 전부를 추출
        trs = table.findAll("tr")

        #처음 값은 쓰레기 값이라 버림
        for tr in trs[1:]:
            print("평점 번호:"+tr.find("td","ac").text, end=" ")
            print("영화 제목:"+tr.find("a","movie").text, end= " ")
            print("평점:"+tr.find("em").text, end= " ")
            print("작성자:"+tr.find("a","author").string)

            num = tr.find("td","ac").text
            movie = tr.find("a","movie").text
            point = tr.find("em").text
            writer = tr.find("a","author").text

            #하나의 평가마다 리스트를 만들어 리스트에 추가함
            result.append([num,movie,point,writer])
    return result

column = ["번호","영화","점수","작성자"]
result = get_movie_point(1,2)

#판다스의 데이터프레임은 리스트 형태의 객체를 받아서 변환해줌
#만약 딕셔너리 안에 리스트를 가지고 있는 형태로 받는다면 컬럼 지정 가능
#현재는 이중 리스트를 받아왔고 변수를 따로 지정하여 컬럼 지정
#단, 이중리스트로 받아오면 가로축으로 데이터가 출력되고,
#딕셔너리로 받아오면 데이터가 세로축으로 출력됨
dataframe = pandas.DataFrame(result, columns=column)
print('-----------------------Data Frame-----------------------')
print(dataframe)

dataframe.to_excel("movie.xlsx",sheet_name="naverMovie",header=True,startrow=1)

page: 1
평점 번호:17952031 영화 제목:캐리비안의 해적 - 망자의 함 평점:10 작성자:mero****
평점 번호:17952030 영화 제목:나일 강의 죽음 평점:10 작성자:vxl4****
평점 번호:17952029 영화 제목:나일 강의 죽음 평점:10 작성자:ounm****
평점 번호:17952028 영화 제목:킹스맨: 퍼스트 에이전트 평점:8 작성자:skqk****
평점 번호:17952027 영화 제목:해적: 도깨비 깃발 평점:1 작성자:ppss****
평점 번호:17952026 영화 제목:나일 강의 죽음 평점:10 작성자:qhwd****
평점 번호:17952025 영화 제목:만년이 지나도 변하지 않는 게 있어 평점:8 작성자:aldl****
평점 번호:17952023 영화 제목:캐리비안의 해적 - 블랙 펄의 저주 평점:10 작성자:mero****
평점 번호:17952024 영화 제목:나일 강의 죽음 평점:10 작성자:8uv5****
평점 번호:17952022 영화 제목:굿 보스 평점:10 작성자:soul****
page: 2
평점 번호:17952020 영화 제목:나의 촛불 평점:1 작성자:king****
평점 번호:17952021 영화 제목:나일 강의 죽음 평점:10 작성자:ajgz****
평점 번호:17952019 영화 제목:나일 강의 죽음 평점:10 작성자:t2l0****
평점 번호:17952018 영화 제목:나일 강의 죽음 평점:10 작성자:vwsk****
평점 번호:17952017 영화 제목:굿 보스 평점:7 작성자:skan****
평점 번호:17952016 영화 제목:나일 강의 죽음 평점:10 작성자:hdbo****
평점 번호:17952015 영화 제목:나일 강의 죽음 평점:5 작성자:jiso****
평점 번호:17952014 영화 제목:해적: 도깨비 깃발 평점:1 작성자:jinn****
평점 번호:17952013 영화 제목:악마는 프라다를 입는다 평점:10 작성자:seco****
평점 번호:17952012 영화 제목:끝까지 간다 